In [139]:
import os
import sys
import torch

from torch import nn
from pprint import pprint
from dataclasses import dataclass,asdict

from graph_bridges.data.transforms import SpinsToBinaryTensor

In [10]:
from graph_bridges.models.spin_glass.ising_parameters import ParametrizedSpinGlassHamiltonianConfig
from graph_bridges.models.spin_glass.ising_parameters import ParametrizedSpinGlassHamiltonian

In [11]:
from graph_bridges.configs.spin_glass.spin_glass_config_sb import SBConfig

In [25]:
number_of_spins = 3
number_of_paths = 20
batch_size = 32
number_of_mcmc_steps = 500
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

config = ParametrizedSpinGlassHamiltonianConfig()
number_of_couplings = ParametrizedSpinGlassHamiltonian.coupling_size(number_of_spins)

#fields = torch.Tensor(size=(number_of_spins,)).normal_(0.,1./number_of_spins)
#couplings = torch.Tensor(size=(number_of_couplings,)).normal_(0.,1/number_of_spins)
fields, couplings = ParametrizedSpinGlassHamiltonian.sample_random_model(number_of_spins)

In [26]:
config.number_of_spins = number_of_spins
config.couplings_deterministic =  None
config.obtain_partition_function = False
config.number_of_mcmc_steps = number_of_mcmc_steps
config.number_of_paths = number_of_paths
config.couplings_sigma = 5.
config.fields = fields
config.couplings = couplings

ising_model_real = ParametrizedSpinGlassHamiltonian(config, device)

In [27]:
config = SBConfig()
config.align_configurations()

In [28]:
ising_model_real.fields

Parameter containing:
tensor([ 3.6454, -0.3805, -3.7722], device='cuda:0', requires_grad=True)

In [29]:
import torch
import torch.nn as nn

class BilinearLayer(nn.Module):
    def __init__(self, input_dim):
        super(BilinearLayer, self).__init__()
        # Define the matrix A as a parameter of the module
        self.A = nn.Parameter(torch.rand(input_dim, input_dim))  # You can initialize it as you like

    def forward(self, x):
        # Calculate the bilinear form x^T A x
        bilinear_form = torch.matmul(x, torch.matmul(self.A, x))
        return bilinear_form

# Example usage:
input_dim = 3  # Change this according to your input dimension
x = torch.tensor([1.0, 2.0, 3.0], requires_grad=True)

# Create the bilinear layer
bilinear_layer = BilinearLayer(input_dim)

# Forward pass to calculate the bilinear form
output = bilinear_layer(x)

# Compute gradients
output.backward()

# Access the gradients of A
gradient_A = bilinear_layer.A.grad

In [154]:
import torch
import unittest

from pprint import pprint


from graph_bridges.models.generative_models.sb import SB
from graph_bridges.configs.config_sb import SBTrainerConfig
from graph_bridges.data.graph_dataloaders_config import EgoConfig
from graph_bridges.data.spin_glass_dataloaders_config import ParametrizedSpinGlassHamiltonianConfig
from graph_bridges.configs.graphs.graph_config_sb import SBConfig
from graph_bridges.configs.config_sb import ParametrizedSamplerConfig, SteinSpinEstimatorConfig

from graph_bridges.models.temporal_networks.mlp.temporal_mlp import TemporalMLPConfig
from graph_bridges.models.backward_rates.ctdd_backward_rate_config import BackwardRateTemporalHollowTransformerConfig
from graph_bridges.models.temporal_networks.transformers.temporal_hollow_transformers import TemporalHollowTransformerConfig
from graph_bridges.models.trainers.sb_training import SBTrainer
from graph_bridges.models.spin_glass.spin_utils import copy_and_flip_spins

from graph_bridges.data.transforms import SpinsToBinaryTensor
spins_to_binary = SpinsToBinaryTensor()

sb_config = SBConfig(delete=True,
                     experiment_name="spin_glass",
                     experiment_type="sb",
                     experiment_indentifier=None)

#self.sb_config.data = EgoConfig(as_image=False, batch_size=2, flatten_adjacency=True,full_adjacency=True)
num_epochs = 200
batch_size = 2

sb_config.data = ParametrizedSpinGlassHamiltonianConfig(data="bernoulli_small",
                                                        batch_size=batch_size,
                                                        number_of_spins=3)
sb_config.target = ParametrizedSpinGlassHamiltonianConfig(data="bernoulli_small",
                                                          batch_size=batch_size,
                                                          number_of_spins=3)

sb_config.temp_network = TemporalMLPConfig(time_embed_dim=12,hidden_dim=250)
sb_config.stein = SteinSpinEstimatorConfig(stein_sample_size=200,stein_epsilon=1e-2)
sb_config.sampler = ParametrizedSamplerConfig(num_steps=20)

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

sb = SB()
sb.create_new_from_config(sb_config,device)

batchdata = next(sb.data_dataloader.train().__iter__())
X_spins = batchdata[0]
X_copy_spin, X_flipped_spin = copy_and_flip_spins(X_spins)
X_spins = X_spins.to(device)
X_copy_spin = X_copy_spin.to(device)
X_flipped_spin = X_flipped_spin.to(device)

fake_time = torch.rand((X_copy_spin.shape[0],)).to(device)
current_model = sb.training_model
phi_0 = current_model(X_copy_spin,fake_time)
phi_1 = current_model(X_flipped_spin,fake_time)

Coupling from Spin File Different from Config
Coupling from Spin File Different from Config
Coupling from Spin File Different from Config
Coupling from Spin File Different from Config


In [155]:
X_copy_spin

tensor([[ 1., -1.,  1.],
        [ 1., -1.,  1.],
        [ 1., -1.,  1.],
        [ 1., -1., -1.],
        [ 1., -1., -1.],
        [ 1., -1., -1.]], device='cuda:0')

In [182]:
import torch

# Define your MLP model
class MLP(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(MLP, self).__init__()
        self.fc1 = torch.nn.Linear(input_dim, hidden_dim)
        self.relu = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x


import torch

def compute_jacobian(model, input_data):
    input_data.requires_grad_(True)
    output = model(input_data)
    batch_size, num_outputs = output.size(0), output.size(1)
    jacobian = torch.zeros(batch_size, num_outputs, input_data.size(1))
    
    for i in range(num_outputs):
        output_i = output[:, i]
        gradient = torch.zeros_like(output_i)
        gradient.fill_(1.0)
        output_i.backward(gradient, retain_graph=True)
        jacobian[:, i, :] = input_data.grad.clone()
        input_data.grad.zero_()
    
    return jacobian

In [183]:
# Create an instance of the MLP
input_dim = 3
output_dim = 1

hidden_dim = 10
batch_size = 23
input_data = torch.randn(batch_size, input_dim)

cpu_device = torch.device("cpu")
X_spins = X_spins.to(torch.device("cpu"))
mlp = MLP(input_dim, hidden_dim, output_dim)
spins = spins_to_binary(X_spins)
copy_spin = spins_to_binary(X_copy_spin).to(cpu_device)
flipped_spin = spins_to_binary(X_flipped_spin).to(cpu_device)

In [184]:
J = compute_jacobian(mlp, spins)
J = J.squeeze()

In [185]:
J

tensor([[-0.1692,  0.0123,  0.0648],
        [-0.1697,  0.0094,  0.0722]])

In [186]:
-(2.*spins - 1.)*J

tensor([[ 0.1692,  0.0123, -0.0648],
        [ 0.1697,  0.0094,  0.0722]], grad_fn=<MulBackward0>)

In [187]:
mlp(flipped_spin) - mlp(copy_spin)

tensor([[ 0.0799],
        [ 0.0142],
        [-0.0680],
        [ 0.1733],
        [ 0.0166],
        [ 0.0680]], grad_fn=<SubBackward0>)

In [188]:
copy_spin

tensor([[1., 0., 1.],
        [1., 0., 1.],
        [1., 0., 1.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.]])

In [179]:
flipped_spin

tensor([[0., 0., 1.],
        [1., 1., 1.],
        [1., 0., 0.],
        [0., 0., 0.],
        [1., 1., 0.],
        [1., 0., 1.]])

In [ ]:
import torch
import torch.nn.functional as F

def gibbs_with_gradients(unnormalized_log_prob, current_sample, num_samples=1):
    samples = []
    x = current_sample.clone()  # Make a copy of the current sample
    
    for _ in range(num_samples):
        # Compute d_tilde(x)
        d_tilde = torch.sigmoid(x)  # For binary data, Eq. 3
        
        # Compute q(i|x)
        q_i_given_x = F.softmax(d_tilde, dim=0)
        
        # Sample i ~ q(i|x)
        i = torch.multinomial(q_i_given_x, 1).item()
        
        # Flip dimension i
        x_flipped = x.clone()
        x_flipped[i] = 1 - x[i]
        
        # Compute d_tilde(x_flipped)
        d_tilde_flipped = torch.sigmoid(x_flipped)  # For binary data, Eq. 3
        
        # Compute q(i|x_flipped)
        q_i_given_x_flipped = F.softmax(d_tilde_flipped, dim=0)
        
        # Compute the acceptance probability
        acceptance_prob = min(1, torch.exp(unnormalized_log_prob(x_flipped) - unnormalized_log_prob(x)) * 
                                   q_i_given_x_flipped[i] / q_i_given_x[i])
        
        # Accept the new sample with probability acceptance_prob
        if torch.rand(1).item() < acceptance_prob:
            x = x_flipped
        
        samples.append(x.clone())
    
    return torch.stack(samples)

# Example usage:
# Define your unnormalized log-prob function unnormalized_log_prob(x)
# Initialize a current sample x_current
# Call the gibbs_with_gradients function to perform Gibbs sampling
# sampled_samples = gibbs_with_gradients(unnormalized_log_prob, x_current, num_samples=1000)


In [189]:
import torch

def sigmoid(x):
    return 1 / (1 + torch.exp(-x))

def contrastive_divergence(rbm, input_data, k=1, learning_rate=0.1, num_epochs=100):
    num_samples, num_visible_units = input_data.size()
    num_hidden_units = rbm.num_hidden_units
    
    for epoch in range(num_epochs):
        positive_associations = torch.matmul(input_data.t(), rbm.sample_hidden_given_visible(input_data))
        negative_visible_activations = rbm.sample_visible_given_hidden(rbm.sample_hidden_given_visible(input_data))
        negative_hidden_activations = rbm.sample_hidden_given_visible(negative_visible_activations)
        negative_associations = torch.matmul(negative_visible_activations.t(), negative_hidden_activations)
        
        rbm.weights += learning_rate * ((positive_associations - negative_associations) / num_samples)
        rbm.visible_bias += learning_rate * torch.sum(input_data - negative_visible_activations, dim=0) / num_samples
        rbm.hidden_bias += learning_rate * torch.sum(rbm.sample_hidden_given_visible(input_data) - negative_hidden_activations, dim=0) / num_samples
        
        if epoch % 10 == 0:
            reconstruction_error = torch.mean(torch.square(input_data - negative_visible_activations))
            print(f"Epoch {epoch}: Reconstruction Error = {reconstruction_error.item()}")

class RBM:
    def __init__(self, num_visible_units, num_hidden_units):
        self.num_visible_units = num_visible_units
        self.num_hidden_units = num_hidden_units
        self.weights = torch.randn(num_visible_units, num_hidden_units)
        self.visible_bias = torch.zeros(num_visible_units)
        self.hidden_bias = torch.zeros(num_hidden_units)
    
    def sample_hidden_given_visible(self, visible):
        hidden_prob = sigmoid(torch.matmul(visible, self.weights) + self.hidden_bias)
        return torch.bernoulli(hidden_prob)
    
    def sample_visible_given_hidden(self, hidden):
        visible_prob = sigmoid(torch.matmul(hidden, self.weights.t()) + self.visible_bias)
        return torch.bernoulli(visible_prob)

# Example usage:
# Define the number of visible and hidden units
num_visible_units = 6
num_hidden_units = 2

# Create an RBM model
rbm = RBM(num_visible_units, num_hidden_units)

# Generate some example training data
input_data = torch.randint(2, size=(100, num_visible_units), dtype=torch.float32)

# Train the RBM using Contrastive Divergence
contrastive_divergence(rbm, input_data, k=1, learning_rate=0.1, num_epochs=1000)


Epoch 0: Reconstruction Error = 0.4866666793823242
Epoch 10: Reconstruction Error = 0.4416666626930237
Epoch 20: Reconstruction Error = 0.4933333396911621
Epoch 30: Reconstruction Error = 0.48500001430511475
Epoch 40: Reconstruction Error = 0.4266666769981384
Epoch 50: Reconstruction Error = 0.4566666781902313
Epoch 60: Reconstruction Error = 0.45500001311302185
Epoch 70: Reconstruction Error = 0.4483333230018616
Epoch 80: Reconstruction Error = 0.4749999940395355
Epoch 90: Reconstruction Error = 0.43833333253860474
Epoch 100: Reconstruction Error = 0.4483333230018616
Epoch 110: Reconstruction Error = 0.44999998807907104
Epoch 120: Reconstruction Error = 0.4583333432674408
Epoch 130: Reconstruction Error = 0.46666666865348816
Epoch 140: Reconstruction Error = 0.47833332419395447
Epoch 150: Reconstruction Error = 0.47333332896232605
Epoch 160: Reconstruction Error = 0.4650000035762787
Epoch 170: Reconstruction Error = 0.46166667342185974
Epoch 180: Reconstruction Error = 0.4499999880790